<a href="https://colab.research.google.com/github/autumnjohnson/ceti_audio/blob/main/load_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/autumnjohnson/ceti_audio.git